In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should return your GPU name


True
NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
from langchain_core.tools import tool
from foodnutients import find_best_match, get_food_details, find_best_food_match

ModuleNotFoundError: No module named 'foodnutients'

In [2]:
from services.load_food_embedding import df_recipes, food_embeddings, food_names, model

ModuleNotFoundError: No module named 'services'

In [3]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]

In [4]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

In [5]:
from langchain_ollama import ChatOllama

local_llm = "llama3.1:8b"
llm = ChatOllama(model=local_llm, temperature=0)

In [6]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

In [7]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': '7adbd67e-6f4b-45dc-a001-f34c5d19d7eb',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': '02a0710b-446c-42a9-aa8e-9698c551db65',
  'type': 'tool_call'}]

In [8]:
async for chunk in llm_with_tools.astream(query):
    print(chunk.tool_call_chunks)

[{'name': 'multiply', 'args': '{"a": 3, "b": 12}', 'id': '7c4eba8d-f18c-4fa8-9dd5-65f3d11a355e', 'index': None, 'type': 'tool_call_chunk'}]
[{'name': 'add', 'args': '{"a": 11, "b": 49}', 'id': '97b153cd-6a68-43c8-a919-39cb2c31cc4b', 'index': None, 'type': 'tool_call_chunk'}]
[]


In [ ]:
from typing import Annotated

from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_ollama import ChatOllama



In [ ]:

class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

tool = {
    "name": "calculator",
    "description": "A simple calculator that can perform basic arithmetic operations.",
    "parameters": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "description": "The operation to perform. Can be 'add', 'subtract', 'multiply', or 'divide'.",
                "enum": ["add", "subtract", "multiply", "divide"],
            },
            "a": {"type": "number", "description": "The first number."},
            "b": {"type": "number", "description": "The second number."},
        },
        # Required parameters for the tool
        "required": ["operation", "a", "b"],
    },
    "return_type": {
        "type": "object",
        "properties": {
            "result": {
                "type": "number",
                "description": "The result of the operation.",
            },
        },
    },
    "return_description": "The result of the operation.",
    "return_example": {
        "result": 5,
    },
}
tools = [tool]
local_llm = "llama3:latest"
llm = ChatOllama(model=local_llm, temperature=0)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")